In [1]:
# file related
import os
from os.path import join

# machine learning
from keras.models import Sequential
from keras.layers import Dense, Input
import tensorflow as tf
from sklearn.model_selection import train_test_split

# utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import librosa
import import_ipynb
from IPython.display import Audio as play_audio
import scipy as sp
import matplotlib.patches as patches
import taunet_utils
%run transient_shaper_lib.ipynb

print(f"TF version {tf.__version__}")

TF version 2.12.0


In [2]:
def create_dataset(audio_dir, human_input_csv):
    lib = taunet_utils.af_dsp_init("../src/af/AFInC.dll")
    audio_data = taunet_utils.read_audio_files(audio_dir)
    human_data = pd.read_csv(human_input_csv)

    input_data = dict()
    output_data = dict()
    
    for audio, label, fs in audio_data:
        lib.initAf()
        lib.resetBuffer()
        label = label.split()[0]
        for sample in audio:
            lib.AFInCAppend(sample)
        lib.AFInCProcess()
        
        human_input = human_data[human_data['MEASUREMENT_ID'] == int(label)].iloc[0].to_dict()
        human_input.pop("MEASUREMENT_ID")
        human_input.pop("SONG_ID")
        human_output = dict()
        human_output["ATTACK_T1"] = human_input.pop("ATTACK_T1")
        human_output["SUSTAIN_T1"] = human_input.pop("SUSTAIN_T1")
        output_data[label] = tuple(human_output.values())
        
        input_data[label] = tuple(round(val, 4) for val in (
            lib.afGetTempo(),
            lib.afGetT1A() / fs,
            lib.afGetT2A() / fs,
            int(lib.afGetSpectralCentroid()),
            lib.afGetSpectralFlatness(),
            int(lib.afGetPBandL()),
            int(lib.afGetPBandML()),
            int(lib.afGetPBandMH()),
            int(lib.afGetPBandH()),
            lib.afGetCrestFactor()
        ))
        input_data[label] += tuple(human_input.values())
    return input_data, output_data

input_data, output_data = create_dataset(os.path.join("dataset", "audio"), os.path.join("dataset", "human_input", "AITD_Dataset_Kristof_beta_1.csv"))
for d in input_data:
    print(f"{d}: {input_data[d]} -> {output_data[d]}")


1: (123.9669, 0.012, 0.2251, 15648, 0.7536, 10, 1, 1, 1, -2097225378, 0.6738, 0.8758, 550.0, 350.0) -> (360.5123, 1144.7427)
10: (159.0106, 0.0131, 0.2123, 2440, 0.8535, 10, 1, 1, 1, 0, 0.3157, 0.8187, 550.0, 350.0) -> (330.558, 1814.8958)
11: (117.4935, 0.0408, 0.31, 7351, 0.7748, 10, 1, 1, 1, 546576167, 0.7365, 0.7567, 550.0, 350.0) -> (302.5441, 430.5534)
12: (172.4138, 0.3676, 0.4617, 15840, 1.569, 10, 1, 1, 1, 102581079, 0.7287, 0.8574, 550.0, 350.0) -> (70.1614, 1192.9986)
13: (156.7944, 0.0098, 0.1759, 3139, 0.859, 10, 1, 1, 1, 78035367, 0.109, 0.9997, 550.0, 350.0) -> (283.9686, 1571.1926)
14: (152.5424, 0.0422, 0.245, 2145, 1.1746, 10, 1, 1, 1, 0, 0.6313, 0.2773, 550.0, 350.0) -> (281.9786, 461.7476)
15: (143.77, 1.4763, 0.1339, 2145, 2.2429, 10, 1, 1, 1, -1248480519, 0.5662, 0.8151, 550.0, 350.0) -> (206.762, 1754.8867)
16: (127.8409, 0.0328, 0.2358, 2134, 0.8128, 10, 1, 1, 1, 1320852491, 0.6062, 0.2058, 550.0, 350.0) -> (499.6292, 320.9579)
17: (140.1869, 0.0419, 0.1801, 927

In [5]:
def create_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    # model.add(layers.LSTM(64, return_sequences=True))   # short term memory, useful if input data is related accross vectors
    model.add(Dense(2)) # no activation (linear): continuous mapping of outputs (this is not a classification task!)
    return model

input_shape = (14,)
model = create_model(input_shape)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                960       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 16)                528       
                                                                 
 dense_6 (Dense)             (None, 8)                 136       
                                                                 
 dense_7 (Dense)             (None, 2)                 18        
                                                                 
Total params: 3,722
Trainable params: 3,722
Non-trainable params: 0
_________________________________________________________________


In [9]:
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Step 2: Combine Data
combined_data = [(input_data[key], output_data[key]) for key in input_data.keys()]

# Step 3: Split Data
train_data, test_data = train_test_split(combined_data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Step 4: TensorBoard Integration
# Define TensorBoard log directory
log_dir = "logs/"

# Define TensorBoard callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Step 5: Logging
def log_metrics(metrics_dict, prefix=""):
    for metric_name, metric_value in metrics_dict.items():
        logger.info(f"{prefix}{metric_name}: {metric_value}")

%load_ext tensorboard
%tensorboard --logdir logs/ --port 6006

# Train the model
history = model.fit(
    x=np.array([item[0] for item in train_data]),
    y=np.array([item[1] for item in train_data]),
    validation_data=(
        np.array([item[0] for item in val_data]),
        np.array([item[1] for item in val_data])
    ),
    epochs=100,
    callbacks=[tensorboard_callback]
)

# Log metrics after training
train_metrics = model.evaluate(
    np.array([item[0] for item in train_data]),
    np.array([item[1] for item in train_data]),
    verbose=0
)
val_metrics = model.evaluate(
    np.array([item[0] for item in val_data]),
    np.array([item[1] for item in val_data]),
    verbose=0
)

log_metrics({f"Train {metric_name}": metric_value for metric_name, metric_value in zip(model.metrics_names, train_metrics)}, "Final ")
log_metrics({f"Validation {metric_name}": metric_value for metric_name, metric_value in zip(model.metrics_names, val_metrics)}, "Final ")

model.save("taunet.h5")

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 11700), started 0:10:50 ago. (Use '!kill 11700' to kill it.)

Epoch 1/100
2/2 [==============================] - 1s 580ms/step - loss: 114867617792.0000 - accuracy: 0.7027 - val_loss: 4027687424.0000 - val_accuracy: 0.3000
Epoch 2/100
2/2 [==============================] - 0s 338ms/step - loss: 110603075584.0000 - accuracy: 0.7027 - val_loss: 3861800448.0000 - val_accuracy: 0.3000
Epoch 3/100
2/2 [==============================] - 0s 341ms/step - loss: 105787047936.0000 - accuracy: 0.7027 - val_loss: 3698387456.0000 - val_accuracy: 0.3000
Epoch 4/100
2/2 [==============================] - 0s 401ms/step - loss: 101431320576.0000 - accuracy: 0.7027 - val_loss: 3531942400.0000 - val_accuracy: 0.3000
Epoch 5/100
2/2 [==============================] - 0s 287ms/step - loss: 96714817536.0000 - accuracy: 0.7027 - val_loss: 3637867264.0000 - val_accuracy: 0.3000
Epoch 6/100
2/2 [==============================] - 1s 512ms/step - loss: 99150282752.0000 - accuracy: 0.7027 - val_loss: 3350287872.0000 - val_accuracy: 0.3000
Epoch 7/100
2/2 [===================

INFO:__main__:Final Train loss: 5291149824.0
INFO:__main__:Final Train accuracy: 0.7027027010917664
INFO:__main__:Final Validation loss: 200539824.0
INFO:__main__:Final Validation accuracy: 0.30000001192092896
